In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
import numpy as np
import pyaudio
import time
import librosa
import wave
import queue
import IPython.display as ipd
from IPython.display import display, Audio
import soundfile as sf
import os
import threading

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


In [2]:
# Định nghĩa các hằng số
RATE = 16000
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
n_times_1s = int(RATE / CHUNK * 1)  # Number of iterations to record 1 second
n_times_145s = int(RATE* 1.5 / CHUNK )  # Number of iterations to record 1 second
predict_mode = False
p = pyaudio.PyAudio()

In [3]:
def features_extractor_1s(file_name):
    audio,sample_rate  = librosa.load(file_name, sr=16000, mono=True)
    mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40) ## Apllying mfcc
    mfcc_processed = np.mean(mfcc.T, axis=0) ## some pre-processing
    return mfcc_processed

In [4]:
# def features_extractor_145s(file_name):
#     audio,sample_rate  = librosa.load(file_name, sr=16000, mono=True)
#     mel = mfcc(audio,sample_rate, nfilt=26, numcep=26, nfft=512)
#     imarray = np.resize(mel, (250, 26))
#     return imarray

In [5]:
# Hàm trích xuất đặc trưng MFCC
def features_extractor_145s(file_name):
    audio, sample_rate = librosa.load(file_name,sr=16000, mono=True)
    frame_length = int(0.025 * sample_rate)
    hop_length = int(0.015 * sample_rate)
    n_fft = 2 ** int(np.ceil(np.log2(frame_length)))
    mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=13)
    desired_size = 101
    mfccs = np.zeros((13,desired_size))
    mfccs[:, :min(desired_size, mfcc.shape[1])] = mfcc[:, :min(desired_size, mfcc.shape[1])]
    delta_mfccs = librosa.feature.delta(mfccs)
    delta2_mfccs = librosa.feature.delta(mfccs, order=2)
    mfccs_features = np.concatenate((mfccs, delta_mfccs, delta2_mfccs))
    mfccs_features = np.transpose(mfccs_features)

    return mfccs_features

In [6]:
def mic_check_1s(file_name):
    return 0

In [7]:
# Hàm dự đoán số 
def mic_check_145s(file_name):
    global predict_mode
    mfccs_features = features_extractor_145s(file_name)
    # print(mfccs_features.shape())
    # data = mfccs_features
    data = np.stack(mfccs_features)
    data = np.reshape(data,(1,101,39))
    pre_test = model_145s.predict(data, verbose=0)
    print(pre_test)
    s=0
    result = 0
    for i in pre_test[0]:
        if i > 9e-1:
            result = i
            predict_mode = True
            break
        s=s+1
    if s < 5:
        time.sleep(1)
    if s == 0:
        print("Predict: Batdieuhoa")
        display(Audio(get_file_path('Response','Batdieuhoa.wav'), rate=16000, autoplay=True))
    elif s == 1:
        print("Predict: Tatdieuhoa")
        display(Audio(get_file_path('Response','Tatdieuhoa.wav'), rate=16000, autoplay=True))
    elif s == 2:
        print("Predict: Tang1do")
        display(Audio(get_file_path('Response','Tang1do.wav'),  rate=16000,autoplay=True))
    # elif s == 3:
    #     print("Predict: Tang2do")
    # elif s == 4:
    #     print("Predict: Tang3do")
    elif s == 3:
        print("Predict: Giam1do")
        display(Audio(get_file_path('Response','Giam1do.wav'), rate=16000, autoplay=True))
    # elif s == 5:
    #     print("Predict: Giam2do")
    # elif s == 7:
    #     print("Predict: Giam3do")
    elif s == 4:
        print("Predict: Bat26do")
        display(Audio(get_file_path('Response','Bat26do.wav'), rate=16000, autoplay=True))
    elif s == 5:
        print("Predict: Silent")
    else:
        print("Unrecognizable")
    print('Reliability: %.2f %%' %(result*100.00))
    display(Audio(file_name))
    if s < 5: 
        time.sleep(1)
    
    
#         print('Reliability: %.2f %%' %(result*100.00))
#         display(Audio(file_name))
    

In [8]:
# Hàm của luồng 1
def record_audio_1s(file_number, stream):
    print(f"Recording audio for file {file_number}...")
    global predict_mode
    frames = []

    for _ in range(n_times_1s):
        # Lấy một khung dữ liệu
        data = stream.read(CHUNK)
        frames.append(data)

    # Lưu file âm thanh vào file wav
    filename =  "wakeup_{}".format(file_number)+".wav"
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
    return filename
        

In [9]:
# Hàm của luồng 1
def record_audio_145s(file_number, stream):
    print(f"Recording audio for file {file_number}...")
    frames = []

    for _ in range(n_times_145s):
        # Lấy một khung dữ liệu
        data = stream.read(CHUNK)
        frames.append(data)

    # Lưu file âm thanh vào file wav
    filename =  "speech_{}".format(file_number)+".wav"
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
    return filename

In [10]:
# Hàm của luồng 1
def record_audio():
    print("Recording audio...")
    global predict_mode
    #Khởi tạo Audio
    stream = p.open(format=FORMAT,
                    channels=1,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)
    for i in range(100):
        speech = record_audio_145s(i, stream)
        mic_check_145s(speech)
        !del {speech}
    
    # Đóng stream và thoát khỏi PyAudio
    stream.stop_stream()
    stream.close()
    p.terminate()

    
      
    

In [11]:

def get_file_paths_in_folder(folder_path):
    # Kiểm tra xem đường dẫn thư mục có tồn tại không
    if not os.path.exists(folder_path):
        return "Thư mục không tồn tại"
    
    # Lấy danh sách tên file trong thư mục
    files = os.listdir(folder_path)
    
    # Lọc ra chỉ các tên file (không phải thư mục) và kết hợp với đường dẫn thư mục
    file_paths = [os.path.join(folder_path, file_name) for file_name in files if os.path.isfile(os.path.join(folder_path, file_name))]
    
    return file_paths

In [12]:
def get_file_path(folder,file_name):
    # Lấy đường dẫn đến thư mục làm việc hiện tại trong Jupyter
    current_directory = os.getcwd()

    # Đường dẫn đến thư mục chứa tệp mô hình "H5"
    h5_folder_path = os.path.join(current_directory, folder)

    # Đường dẫn đến tệp mô hình được chỉ định
    file_path = os.path.join(h5_folder_path, file_name)

    return file_path


In [13]:
# model_145s = load_model("add_x3_bg_9_512.h5", compile=False)
# model_145s = load_model("check_5.h5", compile=False)
# model_145s = load_model("check_7_2000.h5", compile=False)


# model_145s = load_model(get_file_path("H5","AN_Rain.h5"), compile=False)

model_145s = load_model(get_file_path("H5","Train_101_True_AIS_150_29_LSTM.h5"), compile=False)
# Au_AN_Rain


In [ ]:
record_audio()

Recording audio...
Recording audio for file 0...
[[1.0297944e-20 3.3618152e-14 1.0791167e-12 1.1969444e-14 1.7100072e-14
  1.0000000e+00]]
Predict: Silent
Reliability: 100.00 %


Recording audio for file 1...
[[2.8188102e-32 1.3027352e-21 1.0758863e-19 1.0869242e-21 2.5710434e-25
  1.0000000e+00]]
Predict: Silent
Reliability: 100.00 %


Recording audio for file 2...
[[1.4970211e-29 1.5781834e-18 9.4153379e-15 7.1376522e-18 3.9974901e-21
  1.0000000e+00]]
Predict: Silent
Reliability: 100.00 %


Recording audio for file 3...
[[7.1618588e-29 5.0835282e-18 1.7739876e-13 6.3338728e-17 2.2189565e-19
  1.0000000e+00]]
Predict: Silent
Reliability: 100.00 %


Recording audio for file 4...
[[1.2277440e-28 7.4579431e-18 3.2501311e-12 6.8835715e-16 9.3190912e-19
  1.0000000e+00]]
Predict: Silent
Reliability: 100.00 %


...

Recording audio for file 9...
[[4.3869217e-27 3.0521423e-17 2.2276395e-13 1.3885744e-15 1.2065735e-18
  1.0000000e+00]]
Predict: Silent
Reliability: 100.00 %


Recording audio for file 10...
[[1.8190107e-31 2.8363878e-20 2.5977225e-16 2.1458686e-18 1.6033876e-22
  1.0000000e+00]]
Predict: Silent
Reliability: 100.00 %


Recording audio for file 11...
[[1.7017423e-26 2.7938413e-20 1.3093185e-18 1.2216441e-18 1.0086493e-18
  1.0000000e+00]]
Predict: Silent
Reliability: 100.00 %


Recording audio for file 12...
[[5.5156443e-22 8.9921632e-16 5.7946662e-17 2.4693240e-18 1.7726596e-19
  1.0000000e+00]]
Predict: Silent
Reliability: 100.00 %


Recording audio for file 13...


In [ ]:
# record_audio()